# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [39]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [40]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [41]:
# Import the necessary CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col = "year")

# check for NA values
number_of_na = sfo_data.isna().sum().sum()
print(f"Number of NA values in the SFO Census DataFrame: {number_of_na}")

# check for null values
number_of_nulls = sfo_data.isnull().sum().sum()
print(f"Number of NULL values in the SFO Census DataFrame: {number_of_nulls}")

# Dropping NA and NULL values
sfo_data.dropna(inplace = True)

number_of_na = sfo_data.isna().sum().sum()
print(f"Number of NA values after drop function is applied: {number_of_na}")

# Check for NULL values after drop function
number_of_nulls = sfo_data.isnull().sum().sum()
print(f"Number of NULL values after drop function is applied: {number_of_nulls}")

Number of NA values in the SFO Census DataFrame: 5
Number of NULL values in the SFO Census DataFrame: 5
Number of NA values after drop function is applied: 0
Number of NULL values after drop function is applied: 0


In [42]:
# Load in neighborhoods coordinates data and setup a DataFrame
file_path = Path("Data/neighborhoods_coordinates.csv")

coordinate_data_df = pd.read_csv(file_path)
coordinate_data_df.columns = ["neighborhood", "lat", "lon"]
coordiante_data_df = coordinate_data_df.set_index("neighborhood")

# checking for NA values
number_of_na_coordiante_data = coordinate_data_df.isna().sum().sum()
print(f"Number of NA values in the coordinate data: {number_of_na_coordiante_data}")

# checking for NULL values
number_of_nulls_coordinate_data = coordinate_data_df.isnull().sum().sum()
print(f"Number of NULL values in the coordinate data: {number_of_nulls_coordinate_data}")

Number of NA values in the coordinate data: 0
Number of NULL values in the coordinate data: 0


# Global DataFrames (used in multiple graphs)

In [45]:
# Create global DataFrames to improve code preformance these DF's are used in multiple graphs to plot the data

# group by year and neighborhood and create a DataFrame with all of the mean values
means = sfo_data.groupby(["year", "neighborhood"]).mean()
means_df = pd.DataFrame(means)

# make a dataframe for sales price per neighborhood, housing units and gross rent for each neighborhood (mean)
mean_values_neighborhood = sfo_data.groupby(["neighborhood"]).mean()
mean_values_neighborhood_df = pd.DataFrame(mean_values_neighborhood)

# filter out the 10 most expensive neighborhoods for sales price per sq ft
top_10_mean_sale_price_of_neighborhood_df = mean_values_neighborhood_df.nlargest(10, "sale_price_sqr_foot")

# reset the index for the top 10 neighborhoods per sq foot dataframe
top_10__mean_sale_price_of_neighborhood_no_index_df = top_10_mean_sale_price_of_neighborhood_df.reset_index()

# list the top 10 neighborhoods
top_ten_neighborhoods = top_10__mean_sale_price_of_neighborhood_no_index_df["neighborhood"].tolist()

# reset and create a dataframe from the original DF (sfo_data) without an index
sfo_data_no_index = sfo_data.reset_index()

# selecting a subject containing the top 10 neighborhood names from (sfo_data_no_index)
yearly_change_of_rent_and_price_top_10_no_index = sfo_data_no_index[sfo_data_no_index["neighborhood"].isin(top_ten_neighborhoods)]

# setting the index of the top 10 neighborhood data with yearly variation included in the neighborhoods
yearly_change_of_rent_and_price_top_10 = yearly_change_of_rent_and_price_top_10_no_index.set_index(["neighborhood"])

# join the average values with the neighborhood locations
combined_sfo_df = pd.concat([mean_values_neighborhood_df, coordinate_data_df], axis = "columns", join = "inner")
combined_sfo_df.head(3)

# remove the index of the combined dataframe
combined_sfo_no_index = combined_sfo_df.reset_index()

# print and view the dataframe
combined_sfo_no_index.head()

,index,sale_price_sqr_foot,housing_units,gross_rent,neighborhood,lat,lon


## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [ ]:
# Define Panel Visualization Functions (calculate on the fly)
def housing_units_per_year_optional():
    """Housing Units Per Year."""
    
    average_housing_units_per_year_optional = sfo_data["housing_units"].groupby("year").mean()
    average_housing_units_per_year_optional_df = pd.DataFrame(average_housing_units_per_year_optional)

# optional challenge
    min_of_annual_unit_averages = int(average_housing_units_per_year_optional_df.min())
    max_of_annual_unit_averages = int(average_housing_units_per_year_optional_df.max())
    std_of_annual_unit_averages = int(average_housing_units_per_year_optional_df.std())

# calculate the yaxsis upper limit = max_height_annual_unit_averages yaxis lower limit = min_height_annual_unit_averages and convert to intergers
    max_height_annual_unit_averages = max_of_annual_unit_averages = std_of_annual_unit_averages
    min_height_annual_unit_averages = min_of_annual_unit_averages - std_of_annual_unit_averages
    
# plot the bar graph with the calculated yaxis range
    housing_units_per_year_plot_optional = average_housing_units_per_year_optional_df.hvplot.bar(
        height = 400,
        width = 800,
        x = "year"
        y = "housing_units",
        xlabel = "Year",
        ylabel = "Average Housing Units",
        ylim = (min_height_annual_unit_averages, max_height_annual_unit_averages),
        title = "Average Annual Housing Unit Occupation in San Francisco Optional"
    )
    return housing_units_per_year_plot_optional
    
    
def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    average_monthly_rent = sfo_data["gross_rent"].groupby(["year"]).mean()
    average_monthly_rent_df = pd.DataFrame(average_monthly_rent)
    
    # line chart for average monthly rent
    average_gross_rent_plot = average_monthly_rent_df.hvplot.line(
        height = 400,
        width = 800,
        x = "year",
        y = "gross_rent",
        xlabel = "Year",
        ylim = (800, 5000),
        title = "Annual Variation o")


def average_sales_price():
    """Average Sales Price Per Year."""
    
    # YOUR CODE HERE!



def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    # YOUR CODE HERE!



def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""

    # YOUR CODE HERE!


def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    
    # YOUR CODE HERE!

    
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""

    # YOUR CODE HERE!



def parallel_categories():
    """Parallel Categories Plot."""
    
    # YOUR CODE HERE!



def neighborhood_map():
    """Neighborhood Map."""

    # YOUR CODE HERE!


def sunburst():
    """Sunburst Plot."""
    
    # YOUR CODE HERE!


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [ ]:
# Create a Title for the Dashboard
occupation_rent_column = pn.Column(
    "##"
)

# Create a tab layout for the dashboard
# YOUR CODE HERE!


# Create the dashboard
# YOUR CODE HERE!

## Serve the Panel Dashboard

In [ ]:
# Serve the# dashboard
# YOUR CODE HERE!

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# housing_units_per_year()

In [ ]:
# average_gross_rent()

In [ ]:
# average_sales_price()

In [ ]:
# average_price_by_neighborhood()

In [ ]:
# top_most_expensive_neighborhoods()

In [ ]:
# most_expensive_neighborhoods_rent_sales()

In [ ]:
# neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()